In [1]:
import os
import glob as glob
import matplotlib.pyplot as plt
import random
import cv2

In [2]:
#if not os.path.exists('yolov7'):
#    !git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...


In [ ]:
#!pip install -r requirements.txt
#!pip uninstall wandb -y

In [9]:
import pandas as pd
df = pd.read_csv('train.csv')
df.head()

,class,image_path,name,xmax,xmin,ymax,ymin
0,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,797.0,701.0,262.0,211.0
1,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,932.0,786.0,329.0,238.0
2,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,736.0,657.0,275.0,229.0
3,7.0,ea906a663da6321bcef78be4b7d1afff.jpg,BAD_BILLBOARD,986.0,786.0,136.0,0.0
4,8.0,1c7d48005a12d1b19261b8e71df7cafe.jpg,SAND_ON_ROAD,667.0,549.0,228.0,179.0


In [5]:
classes = df[['class','name']].value_counts()
classes

class  name             
3.0    GARBAGE              8597
4.0    CONSTRUCTION_ROAD    2730
2.0    POTHOLES             2625
9.0    CLUTTER_SIDEWALK     2253
7.0    BAD_BILLBOARD        1555
0.0    GRAFFITI             1124
8.0    SAND_ON_ROAD          748
10.0   UNKEPT_FACADE         127
1.0    FADED_SIGNAGE         107
5.0    BROKEN_SIGNAGE         83
6.0    BAD_STREETLIGHT         1
dtype: int64

In [6]:
df.name.unique()

array(['GARBAGE', 'BAD_BILLBOARD', 'SAND_ON_ROAD', 'GRAFFITI', 'POTHOLES',
       'CLUTTER_SIDEWALK', 'CONSTRUCTION_ROAD', 'BROKEN_SIGNAGE',
       'UNKEPT_FACADE', 'FADED_SIGNAGE', 'BAD_STREETLIGHT'], dtype=object)

In [25]:
%%writefile data/data.yaml
names:
- GRAFFITI
- FADED SIGNAGE
- POTHOLES
- GARBAGE
- CONSTRUCTION ROAD
- BROKEN_SIGNAGE
- BAD STREETLIGHT
- BAD BILLBOARD
- SAND ON ROAD
- CLUTTER_SIDEWALK
- UNKEPT_FACADE

nc: 11
train: ./data/train/images 
val: ./data/valid/images
test: ./data/tes

Overwriting data/custom_data.yaml


In [7]:
!wandb login ab7d302b943c078bbd1f2dcb63dbc2fbf591ad58

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Nitiz Khanal/.netrc


In [ ]:
!python train.py --batch 8 --epochs 20 --cfg cfg/training/yolov7.yaml --data data/data.yaml --weights 'yolov7.pt' --device 0


In [27]:
!python test.py --data data/custom_data.yaml  --batch 32 --conf 0.36 --iou 0.65 --device 0 --weights runs/train/yolov7/weights/best.pt --name yolov7_416_val


Traceback (most recent call last):
  File "C:\Users\Nitiz Khanal\Documents\yolov7-custom\yolov7-custom\test.py", line 314, in <module>
    opt.data = check_file(opt.data)  # check file
  File "C:\Users\Nitiz Khanal\Documents\yolov7-custom\yolov7-custom\utils\general.py", line 151, in check_file
    assert len(files), f'File Not Found: {file}'  # assert file was found
AssertionError: File Not Found: data/data.yaml


In [229]:
!python detect.py --source "data/test/images" --weights best60.pt --save-txt --conf=0.3

Namespace(weights=['best60.pt'], source='data/test/images', img_size=640, conf_thres=0.3, iou_thres=0.45, device='', view_img=False, save_txt=True, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

1 GARBAGE, Done. (41.7ms) Inference, (37.1ms) NMS
 The image with the result is saved in: runs\detect\exp12\006e04a34f54d116f6aff2bb31cb320c.jpg
1 GRAFFITI, 2 GARBAGEs, 1 BAD BILLBOARD, Done. (16.6ms) Inference, (4.4ms) NMS
 The image with the result is saved in: runs\detect\exp12\006e70d6a51af3e670bbe712a83aa1c5.jpg
2 GARBAGEs, Done. (18.0ms) Inference, (1.1ms) NMS
 The image with the result is saved in: runs\detect\exp12\0081b25823f23a536c9f1952491db636.jpg
1 CONSTRUCTION ROAD, 1 BAD BILLBO

YOLOR  2023-1-2 torch 1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6143.5MB)

D:\conda\envs\CV\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36535712 parameters, 6194944 gradients, 103.3 GFLOPS


In [230]:
import os
files = [file for file in os.listdir("runs\detect\exp12\labels")]

In [231]:
files = files[1:]

In [232]:
len(files)

1975

In [233]:
pip install pybboxes


Note: you may need to restart the kernel to use updated packages.


(720, 1280, 3)

In [147]:
i =0
W, H = 1920, 1080
import pybboxes as pbx

for file in files:
    if i>2:
        break
    i = i+1
    list = []
    with open(f'runs/detect/exp9/labels/{file}') as f:
        print(file)
        lines = f.readlines()
        for i in range(len(lines)):
            list.append(lines[i].split(' '))
            img = file.split('.')[0] + '.jpg'
            cat = float(list[i][0])
            xcen = float(list[i][1])
            ycen = float(list[i][2])
            w = float(list[i][3])
            h = float(list[i][4])
            xmax = 1920*xcen + 960*w
            ymax = 1080*ycen + 540*w
            xmin = xmax - 1920*w 
            ymin = ymax - 1080*w
            bbox = [xmin, ymin, xmax, ymax]

            print(bbox)
           

006e70d6a51af3e670bbe712a83aa1c5.txt
(699, 495, 854, 600)
(826, 424, 994, 547)
(636, 525, 783, 628)
(1444, 346, 1692, 516)


In [148]:
bbox[0]

1444

In [125]:
import cv2
img = cv2.imread("data/test/images/006e70d6a51af3e670bbe712a83aa1c5.jpg")


In [126]:
def plot_one_box(x, img, color=None, label=None, line_thickness=None, Inverted=False):
    # Plots one bounding box on image img
    tl = line_thickness or 2 
    # line thickness
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl)
    if label:
        tf = tl 
        # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 2, thickness=tf)[0]
        if Inverted == True:
            c1 = c2
            c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        else:
            c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1) 
            # filled
        cv2.putText(img,label,(c1[0], c1[1] - 2),0,tl / 2,[225, 255, 255],thickness=tf,lineType=cv2.LINE_AA,)

In [127]:
plot_one_box([699, 495, 854, 600], img, color=(0, 255, 0), label='cat', line_thickness=2)

In [128]:
cv2.imwrite("labelTest/" + '006e70d6a51af3e670bbe712a83aa1c5.jpg', img)

True

In [234]:
cat_map = [
 "GRAFFITI",
 "FADED SIGNAGE",
 "POTHOLES",
 "GARBAGE",
 "CONSTRUCTION ROAD",
 "BROKEN_SIGNAGE",
 "BAD STREETLIGHT",
 "BAD BILLBOARD",
 "SAND ON ROAD",
 "CLUTTER_SIDEWALK",
 "UNKEPT_FACADE",
]

In [235]:
cat_map[int(3.0)]

'GARBAGE'

In [236]:
images = [file for file in os.listdir("runs\detect\exp12") if file.endswith('.jpg')]
len(images)

2092

In [237]:
images[:10]

['006e04a34f54d116f6aff2bb31cb320c.jpg',
 '006e70d6a51af3e670bbe712a83aa1c5.jpg',
 '0081b25823f23a536c9f1952491db636.jpg',
 '00b48429c1382cc7d1c72926acedbaa7.jpg',
 '00b5fc2d3153637e65a35363295e14ba.jpg',
 '00bce25b1d3a3422bb366a12e3e9ac6a.jpg',
 '00df7d46d661eba176ccc167f2763585.jpg',
 '00f12b9ba667d2c9e60aabf56afc5c6c.jpg',
 '012096618361635b074b072f8af26a97.jpg',
 '01267cf16572d5ec876f0111c5af4526.jpg']

In [238]:
not_detected = [image for image in images if image[:-4] + '.txt' not in files]
len(not_detected)

117

In [239]:
import pandas as pd

In [240]:
df = pd.read_csv('sample_submission.csv')

In [241]:
unDetected = df

In [242]:
import numpy as np


In [ ]:
for image in not_detected:
    new_df = {'class': np.nan, 'image_path': image, 'name': np.nan, 'xmax':0 ,'xmin':0,'ymax':0,'ymin':0  }
    unDetected = unDetected.append(new_df, ignore_index = True)

In [244]:
unDetected

,class,image_path,name,xmax,xmin,ymax,ymin
0,NaN,006e04a34f54d116f6aff2bb31cb320c.jpg,NaN,0,0,0,0
1,NaN,023635a9e76f3c09d07824e19ee83d37.jpg,NaN,0,0,0,0
2,NaN,02710df871b98a873d401e304cdfd48b.jpg,NaN,0,0,0,0
3,NaN,027a4ea75b44bb19e5462d550af31565.jpg,NaN,0,0,0,0
4,NaN,030a78514521188500fa24de227b33f0.jpg,NaN,0,0,0,0
...,...,...,...,...,...,...,...
112,NaN,edbcbf3baf7305d9b5638053c2f60623.jpg,NaN,0,0,0,0
113,NaN,f3c187d799742aba7021b255aacda1a2.jpg,NaN,0,0,0,0
114,NaN,fa13a3a6a7c218664b3692c5b7f05d48.jpg,NaN,0,0,0,0
115,NaN,fb0bf13d7b9d5948e166e5c187996677.jpg,NaN,0,0,0,0


In [ ]:
i =0
W, H = 1920, 1080
import pybboxes as pbx

for file in files:
    list = []
    with open(f'runs/detect/exp12/labels/{file}') as f:
        print(file)
        lines = f.readlines()
        for i in range(len(lines)):
            list.append(lines[i].split(' '))
            img = file.split('.')[0] + '.jpg'
            cat = float(list[i][0])
            xcen = float(list[i][1])
            ycen = float(list[i][2])
            w = float(list[i][3])
            h = float(list[i][4])
            yolo_bbox = (xcen,ycen,w,h)
            bbox = pbx.convert_bbox(yolo_bbox, from_type="yolo", to_type="voc", image_size=(W, H))
            df2 = {'class': cat, 'image_path': img, 'name': cat_map[int(cat)], 'xmax':float(bbox[2]) ,'xmin':float(bbox[0]) ,'ymax':float(bbox[3]) ,'ymin':float(bbox[1])   }
            df = df.append(df2, ignore_index = True)
   


In [246]:
df

,class,image_path,name,xmax,xmin,ymax,ymin
0,7.0,006e70d6a51af3e670bbe712a83aa1c5.jpg,BAD BILLBOARD,1690.0,1448.0,510.0,349.0
1,0.0,006e70d6a51af3e670bbe712a83aa1c5.jpg,GRAFFITI,1692.0,1452.0,506.0,354.0
2,3.0,006e70d6a51af3e670bbe712a83aa1c5.jpg,GARBAGE,784.0,638.0,630.0,525.0
3,3.0,006e70d6a51af3e670bbe712a83aa1c5.jpg,GARBAGE,1000.0,824.0,546.0,422.0
4,3.0,0081b25823f23a536c9f1952491db636.jpg,GARBAGE,1816.0,1696.0,615.0,528.0
...,...,...,...,...,...,...,...
7204,0.0,ffe097c41e8ecfd403bad97ec728ab4c.jpg,GRAFFITI,320.0,185.0,431.0,314.0
7205,2.0,ffe097c41e8ecfd403bad97ec728ab4c.jpg,POTHOLES,1307.0,1024.0,645.0,464.0
7206,0.0,ffe097c41e8ecfd403bad97ec728ab4c.jpg,GRAFFITI,559.0,339.0,437.0,309.0
7207,7.0,ffec05b841277940e40de657722e6892.jpg,BAD BILLBOARD,1920.0,1738.0,442.0,314.0


In [247]:
df.image_path.nunique()

1975

In [248]:
df = df.append(unDetected, ignore_index = True)

C:\Users\Nitiz Khanal\AppData\Local\Temp\ipykernel_5280\1779964425.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(unDetected, ignore_index = True)


In [249]:
df.tail()  

,class,image_path,name,xmax,xmin,ymax,ymin
7321,NaN,edbcbf3baf7305d9b5638053c2f60623.jpg,NaN,0,0,0,0
7322,NaN,f3c187d799742aba7021b255aacda1a2.jpg,NaN,0,0,0,0
7323,NaN,fa13a3a6a7c218664b3692c5b7f05d48.jpg,NaN,0,0,0,0
7324,NaN,fb0bf13d7b9d5948e166e5c187996677.jpg,NaN,0,0,0,0
7325,NaN,fccd79371724e5a9aa63e2e7888dfe65.jpg,NaN,0,0,0,0


In [250]:
df.image_path.nunique()

2092

In [251]:
compression_opts = dict(method='zip',
                        archive_name='submission.csv')  
df.to_csv('submssion.zip', index=False,
          compression=compression_opts) 

TypeError: 'str' object is not callable